# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [111]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Conv2D, AveragePooling2D, Flatten
from keras.layers import MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

In [90]:
df = pd.read_csv('Customer_Churn.csv')

In [91]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Clean Data

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [93]:
for i in df:
    print(df[i].nunique())

7043
2
2
2
2
73
2
3
3
3
3
3
3
3
3
3
2
4
1585
6531
2


In [94]:
df = df.drop(['customerID'], axis=1)

In [95]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', '0')

In [96]:
df['TotalCharges'].value_counts().head()

0        11
20.2     11
19.75     9
20.05     8
19.65     8
Name: TotalCharges, dtype: int64

In [97]:
df['TotalCharges'] = df['TotalCharges'].astype(float)

In [98]:
df.shape

(7043, 20)

In [99]:
df_cat = df[['gender', 'SeniorCitizen', 'Partner', 'Dependents',
             'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
             'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
             'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn'
            ]]

df_num = df[['tenure', 'MonthlyCharges', 'TotalCharges']]

In [100]:
import category_encoders as ce

cat_transform = ce.one_hot.OneHotEncoder(verbose=0, cols=None, drop_invariant=False, return_df=True, handle_missing='value', handle_unknown='value', use_cat_names=False)

In [101]:
df_cat = cat_transform.fit_transform(df_cat)

In [102]:
df_cat.shape

(7043, 44)

In [103]:
scaler = StandardScaler()

df_num = scaler.fit_transform(df_num)

df_num = pd.DataFrame(df_num, columns=['tenure', 'MonthlyCharges', 'TotalCharges'])

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [104]:
df_num.shape, df_cat.shape

((7043, 3), (7043, 44))

In [105]:
df_num.head()

,tenure,MonthlyCharges,TotalCharges
0,-1.277445,-1.160323,-0.992611
1,0.066327,-0.259629,-0.172165
2,-1.236724,-0.362660,-0.958066
3,0.514251,-0.746535,-0.193672
4,-1.236724,0.197365,-0.938874


In [106]:
X = df_cat.merge(df_num, how='outer', left_index=True, right_index=True)

In [107]:
X.head()

,gender_1,gender_2,SeniorCitizen,Partner_1,Partner_2,Dependents_1,Dependents_2,PhoneService_1,PhoneService_2,MultipleLines_1,...,PaperlessBilling_2,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,PaymentMethod_4,Churn_1,Churn_2,tenure,MonthlyCharges,TotalCharges
0,1,0,0,1,0,1,0,1,0,1,...,0,1,0,0,0,1,0,-1.277445,-1.160323,-0.992611
1,0,1,0,0,1,1,0,0,1,0,...,1,0,1,0,0,1,0,0.066327,-0.259629,-0.172165
2,0,1,0,0,1,1,0,0,1,0,...,0,0,1,0,0,0,1,-1.236724,-0.362660,-0.958066
3,0,1,0,0,1,1,0,1,0,1,...,1,0,0,1,0,1,0,0.514251,-0.746535,-0.193672
4,1,0,0,0,1,1,0,0,1,0,...,0,1,0,0,0,0,1,-1.236724,0.197365,-0.938874


## Split data into X,y

In [114]:
Y = df['Churn']

#y = y.replace({'No':0, 'Yes':1})

## Create Model

In [123]:
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(keras.layers.Dense(12, input_dim=47, activation='relu'))
	model.add(keras.layers.Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)



## Grid Search

In [124]:
# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, Y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
4695/4695 [==============================] - 1s 145us/sample - loss: 0.2714 - acc: 0.8988
Epoch 2/20
4695/4695 [==============================] - 0s 100us/sample - loss: 0.0386 - acc: 0.9994
Epoch 3/20
4695/4695 [==============================] - 1s 111us/sample - loss: 0.0098 - acc: 1.0000
Epoch 4/20
4695/4695 [==============================] - 1s 109us/sample - loss: 0.0041 - acc: 1.0000
Epoch 5/20
4695/4695 [==============================] - 1s 111us/sample - loss: 0.0022 - acc: 1.0000
Epoch 6/20
4695/4695 [==============================] - 1s 112us/sample - loss: 0.0013 - acc: 1.0000
Epoch 7/20
4695/4695 [==============================] - 1s 107us/sample - loss: 8.0353e-04 - acc: 1.0000
Epoch 8/20
4695/4695 [==============================] - 0s 105us/sample - loss: 5.3439e-04 - acc: 1.0000
Epoch 9/20
4695/4695 [==============================] - 0s 104us/sample - loss: 3.6780e-04 - acc: 1.0000
Epoch 10/20
4695/4695 [========

In [ ]:
y_pred = model.predict_classes(testX2)

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?